In [3]:
import re
import os
import time
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
import warnings
warnings.filterwarnings("ignore")

In [4]:
if not os.path.exists('./features/'):
    os.mkdir('./features/')
if not os.path.exists('./out/'):
    os.mkdir('./out/')
if not os.path.exists('./weights/'):
    os.mkdir('./weights/')
    
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [2]:
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']

invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']

question_info = pd.read_table(os.path.join(DATA_PATH, 'question_info_0926.txt', header=None)
question_info.columns = ['问题ID',  '问题创建时间' , '问题标题的单字编码序列' , '问题标题的切词编码序列' , '问题描述的单字编码序列',  '问题描述的词编码序列' , '问题绑定的话题ID']
                              
member_info = pd.read_table(os.path.join(DATA_PATH, 'member_info_0926.txt'), header=None)
member_info.columns = ['用户ID','性别','创作关键词的编码序列','创作数量等级','创作热度等级','注册类型','注册平台','访问频率','用户二分类特征A','用户二分类特征B','用户二分类特征C','用户二分类特征D','用户二分类特征E','用户分类特征A','用户分类特征B','用户分类特征C','用户分类特征D','用户分类特征E','用户的盐值分数','用户关注的话题','用户感兴趣的话题']

tmp = member_info['用户感兴趣的话题'].apply(lambda x: re.split('[,:]',x))
member_info['用户感兴趣的话题_T'] = tmp.apply(lambda x : ','.join(x[::2]))
member_info['用户感兴趣的话题_score'] = tmp.apply(lambda x : ','.join(x[1::2]))
                              
oversample = False

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)
    
data = data.merge(member_info, on='用户ID', how='left').merge(question_info, on='问题ID', how='left')                              

In [6]:
tic = time.time()
single_word_vectors = pd.read_table(os.path.join(DATA_PATH, 'single_word_vectors_64d.txt'), header=None)
single_word_vectors.columns = ['单字编码序号','SW']
tmp = single_word_vectors.SW.apply(lambda x: x.split(' '))
tmp = pd.DataFrame(list(tmp), columns=['SW_{}'.format(i) for i in range(1,65)]) 
del single_word_vectors['SW']
single_word_vectors = pd.concat([single_word_vectors, tmp], axis=1)

word_vectors = pd.read_table(os.path.join(DATA_PATH, 'word_vectors_64d.txt'), header=None)
word_vectors.columns = ['词编码序号','W']
tmp = word_vectors.W.apply(lambda x: x.split(' '))
tmp = pd.DataFrame(list(tmp), columns=['W_{}'.format(i) for i in range(1,65)]) 
del word_vectors['W']
word_vectors = pd.concat([word_vectors, tmp], axis=1)

topic_vectors = pd.read_table(os.path.join(DATA_PATH, 'topic_vectors_64d.txt'), header=None)
topic_vectors.columns = ['话题ID编码序号','Topic']
tmp = topic_vectors.Topic.apply(lambda x: x.split(' '))
tmp = pd.DataFrame(list(tmp), columns=['T_{}'.format(i) for i in range(1,65)]) 
del topic_vectors['Topic']
topic_vectors = pd.concat([topic_vectors, tmp], axis=1)

print("Used time: %d s" % (time.time()-tic))

In [9]:
tic = time.time()
vocabulary = dict((single_word_vectors['单字编码序号'][i], i) for i in range(len(single_word_vectors)))

cnt_vct = CountVectorizer(token_pattern='\\w+\\d+', binary=True, lowercase=False, vocabulary=vocabulary)
cnt_vct.fit(question_info['问题标题的单字编码序列'])

A = cnt_vct.transform(question_info['问题标题的单字编码序列'])

B = single_word_vectors.values[:, 1:].astype(float)
B = sparse.csr_matrix(B)

C = A.dot(B)
print(C.shape)

tmp = pd.DataFrame(C.toarray())
tmp.columns = ['SW_w2v_sum_{}'.format(i) for i in range(1,65)]
tmp['问题ID'] = question_info['问题ID']

tmp.to_hdf(os.path.join(FEAT_PATH, 'ques_topic_SW_sum_64.h5'), key='data')

print("Used time: %d s" % (time.time()-tic))

(1829900, 64)


In [12]:
tic = time.time()
vocabulary = dict((single_word_vectors['单字编码序号'][i], i) for i in range(len(single_word_vectors)))

cnt_vct = CountVectorizer(token_pattern='\\w+\\d+', binary=True, lowercase=False, vocabulary=vocabulary)
cnt_vct.fit(question_info['问题描述的单字编码序列'])

A = cnt_vct.transform(question_info['问题描述的单字编码序列'])

B = single_word_vectors.values[:, 1:].astype(float)
B = sparse.csr_matrix(B)

C = A.dot(B)
print(C.shape)

tmp = pd.DataFrame(C.toarray())
tmp.columns = ['ques_describe_SW_w2v_sum_{}'.format(i) for i in range(1,65)]
tmp['问题ID'] = question_info['问题ID']

tmp.to_hdf(os.path.join(FEAT_PATH, 'ques_describe_SW_sum_64.h5'), key='data')

print("Used time: %d s" % (time.time()-tic))

(1829900, 64)


In [15]:
tic = time.time()
vocabulary = dict((word_vectors['词编码序号'][i], i) for i in range(len(word_vectors)))

cnt_vct = CountVectorizer(token_pattern='\\w+\\d+', binary=True, lowercase=False, vocabulary=vocabulary)
cnt_vct.fit(question_info['问题标题的切词编码序列'])

A = cnt_vct.transform(question_info['问题标题的切词编码序列'])

B = word_vectors.values[:, 1:].astype(float)
B = sparse.csr_matrix(B)

C = A.dot(B)
print(C.shape)

tmp = pd.DataFrame(C.toarray())
tmp.columns = ['ques_topic_W_w2v_sum_{}'.format(i) for i in range(1,65)]
tmp['问题ID'] = question_info['问题ID']

tmp.to_hdf(os.path.join(FEAT_PATH, 'ques_topic_W_sum_64.h5') ,key='data')

print("Used time: %d s" % (time.time()-tic))

(1829900, 64)


In [18]:
tic = time.time()
vocabulary = dict((word_vectors['词编码序号'][i], i) for i in range(len(word_vectors)))

cnt_vct = CountVectorizer(token_pattern='\\w+\\d+', binary=True, lowercase=False, vocabulary=vocabulary)
cnt_vct.fit(question_info['问题描述的词编码序列'])

A = cnt_vct.transform(question_info['问题描述的词编码序列'])

B = word_vectors.values[:, 1:].astype(float)
B = sparse.csr_matrix(B)

C = A.dot(B)
print(C.shape)

tmp = pd.DataFrame(C.toarray())
tmp.columns = ['ques_describe_W_w2v_sum_{}'.format(i) for i in range(1,65)]
tmp['问题ID'] = question_info['问题ID']

tmp.to_hdf(os.path.join(FEAT_PATH, 'ques_describe_W_sum_64.h5'), key='data')

print("Used time: %d s" % (time.time()-tic))

(1829900, 64)


In [21]:
vocabulary = dict((topic_vectors['话题ID编码序号'][i], i) for i in range(len(topic_vectors)))

cnt_vct = CountVectorizer(token_pattern='\\w+\\d+', binary=True, lowercase=False, vocabulary=vocabulary)
cnt_vct.fit(member_info['用户关注的话题'])

A = cnt_vct.transform(member_info['用户关注的话题'])

B = topic_vectors.values[:, 1:].astype(float)
B = sparse.csr_matrix(B)

C = A.dot(B)
print(C.shape)

tmp = pd.DataFrame(C.toarray())
tmp.columns = ['用户关注的话题_w2v_sum_{}'.format(i) for i in range(1,65)]
tmp['用户ID'] = member_info['用户ID']

tmp.to_hdf(os.path.join(FEAT_PATH, 'user_watched_topic_w2v_sum_64.h5'), key='data')

(1931654, 64)


In [24]:
vocabulary = dict((topic_vectors['话题ID编码序号'][i], i) for i in range(len(topic_vectors)))

cnt_vct = CountVectorizer(token_pattern='\\w+\\d+', binary=True, lowercase=False, vocabulary=vocabulary)
cnt_vct.fit(member_info['用户感兴趣的话题_T'])

A = cnt_vct.transform(member_info['用户感兴趣的话题_T'])

B = topic_vectors.values[:, 1:].astype(float)
B = sparse.csr_matrix(B)

C = A.dot(B)
print(C.shape)

tmp = pd.DataFrame(C.toarray())
tmp.columns = ['用户感兴趣的话题_w2v_sum_{}'.format(i) for i in range(1,65)]
tmp['用户ID'] = member_info['用户ID']

tmp.to_hdf(os.path.join(FEAT_PATH, 'user_fav_topic_w2v_sum_64.h5'), key='data')

(1931654, 64)


In [27]:
vocabulary = dict((topic_vectors['话题ID编码序号'][i], i) for i in range(len(topic_vectors)))

cnt_vct = CountVectorizer(token_pattern='\\w+\\d+', binary=True, lowercase=False, vocabulary=vocabulary)
cnt_vct.fit(question_info['问题绑定的话题ID'])

A = cnt_vct.transform(question_info['问题绑定的话题ID'])

B = topic_vectors.values[:, 1:].astype(float)
B = sparse.csr_matrix(B)

C = A.dot(B)
print(C.shape)

tmp = pd.DataFrame(C.toarray())
tmp.columns = ['问题绑定的话题ID_w2v_sum_{}'.format(i) for i in range(1,65)]
tmp['问题ID'] = question_info['问题ID']

tmp.to_hdf(os.path.join(FEAT_PATH, 'ques_attach_topic_w2v_sum_64.h5'), key='data')

(1829900, 64)
